In [3]:
import time
import psutil
import os

In [18]:
%run BurrowsWheelerTransform.ipynb
%run BurrowsWheelerTransformImproved.ipynb

In [ ]:
"""Test samples"""

test1 = "Tomorrow_and_tomorrow_and_tomorrow$"
test2 = "It_was_the_best_of_times_it_was_the_worst_of_times$"
test3 = "in_the_jingle_jangle_morning_Ill_come_following_you$"
test4 = "GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTTGATTTGG\
GGTTCAAAGCAGTAATTTGGGGTTCAAAGCAGTATCGACAAATAGTAAATCCATTTGTTCATTCAAAGCAGTAATT\
TGGGGTTATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT$"

In [ ]:
""" Performance measurement and benchmarking """

%timeit BWTViaBWM(test1)
%timeit BWTViaBWM(test2)
%timeit BWTViaBWM(test3)
%timeit BWTViaBWM(test4)

print()

%timeit BWTViaSA(test1)
%timeit BWTViaSA(test2)
%timeit BWTViaSA(test3)
%timeit BWTViaSA(test4)

print()

%timeit ReverseBWT(bwt1)
%timeit ReverseBWT(bwt2)
%timeit ReverseBWT(bwt3)
%timeit ReverseBWT(bwt4)

In [ ]:
!python "./memTest/bwmTest.py" $test1
!python "./memTest/bwmTest.py" $test2
!python "./memTest/bwmTest.py" $test3
!python "./memTest/bwmTest.py" $test4

In [ ]:
!python "./memTest/saTest.py" $test1
!python "./memTest/saTest.py" $test2
!python "./memTest/saTest.py" $test3
!python "./memTest/saTest.py" $test4

In [ ]:
!python "./memTest/reverseTest.py" $bwt1
!python "./memTest/reverseTest.py" $bwt2
!python "./memTest/reverseTest.py" $bwt3
!python "./memTest/reverseTest.py" $bwt4

In [ ]:
!python "./memTest/bwmTestFiles.py" 1
!python "./memTest/bwmTestFiles.py" 2
!python "./memTest/bwmTestFiles.py" 3

In [ ]:
!python "./memTest/saTestFiles.py" 1
!python "./memTest/saTestFiles.py" 2
!python "./memTest/saTestFiles.py" 3

In [2]:
"""Benchmarking Data"""

dataSet = [
    {"file" : "./data/13443_ref_Cara_1.0_chr1c.fa",
     "patterns" : [
     "ATGCATG",
     "TCTCTCTA",
     "TTCACTACTCTCA"
     ]},
    {"file" : "./data/10093_ref_PAHARI_EIJ_v1.1_chrX.fa",
     "patterns" : [
     "ATGATG",
     "CTCTCTA",
     "TCACTACTCTCA"
     ]},
    {"file" : "./data/144034_ref_Pbar_UMD_V03_chrUn.fa",
     "patterns": [
     "CGCGAG",
     "GTCGAAT",
     "GGGCGTCATCGCGCG"
     ]}
]

In [4]:
def BenchmarkSearchViaImprovedSort():
    for data in dataSet:
        file = data.get("file")
        genome = GetWholeGenomeFromFile(file)
        patterns = data.get("patterns")
        
        for pattern in patterns:
            startTime = time.time()
            SearchViaImprovedSort(genome, pattern)
            endTime = time.time()
            duration = endTime - startTime
            print(f"{file} : {pattern} executed in {duration}")
        

In [5]:
BenchmarkSearchViaImprovedSort()

D:\programi\Anaconda\lib\site-packages\pydivsufsort\divsufsort.py:76: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")


./data/13443_ref_Cara_1.0_chr1c.fa : ATGCATG executed in 50.66999697685242
./data/13443_ref_Cara_1.0_chr1c.fa : TCTCTCTA executed in 49.83700251579285
./data/13443_ref_Cara_1.0_chr1c.fa : TTCACTACTCTCA executed in 49.61999797821045
./data/10093_ref_PAHARI_EIJ_v1.1_chrX.fa : ATGATG executed in 372.8929054737091


KeyboardInterrupt: 

In [ ]:
#for i in range(1, len(sequences) + 1):
#    with open("./data/" + "sequence" + str(i) + ".txt", "w") as f:
#        f.write(sequences[i - 1])
#        f.close()

In [19]:
genome = GetWholeGenomeFromFile(dataSet[0].get("file"))
genome[len(genome)-5:len(genome)]

'CTCT$'